<a href="https://colab.research.google.com/github/BrokenShell/DS-Unit-4-Sprint-1-NLP/blob/master/module4-topic-modeling/LS_DS_414_Topic_Modeling_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [ ]:
!pip install pyLDAvis

In [ ]:
import pandas as pd
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric, stem_text
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords, strip_short

import pyLDAvis.gensim

In [ ]:
df = pd.read_csv("Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

In [ ]:
def tokenize(text):
    return [token for token in simple_preprocess(text, max_len=40) if token not in STOPWORDS]

In [ ]:
ex_str = "This is an example of Gensim                 simple_preprocess utility function. "
tokenize(ex_str)

['example', 'gensim', 'simple_preprocess', 'utility', 'function']

In [ ]:
CUSTOM_FILTERS = [lambda x: x.lower(),
                  strip_multiple_whitespaces,
                  strip_numeric,
                  remove_stopwords,
                  strip_non_alphanum,
                  strip_short
                 ]
preprocess_string(ex_str, CUSTOM_FILTERS)

['example', 'gensim', 'simple_preprocess', 'utility', 'function']

In [ ]:
df['tokenize_text'] = df['reviews.text'].apply(preprocess_string, CUSTOM_FILTERS)
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,manufacturer,manufacturerNumber,reviews.date,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,tokenize_text
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2017-03-02T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...","[order, item, bad, qualiti, miss, backup, spri..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2016-08-31T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...","[bulk, expens, wai, product, like]"
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2016-12-06T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...","[duracel, price, happi]"
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2016-03-28T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...","[work, brand, batteri, better, price]"
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2017-04-22T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...","[batteri, long, last, price, great]"


In [ ]:
id2word = corpora.Dictionary(df['tokenize_text'])

In [ ]:
id2word.token2id['duracel']

16

In [ ]:
id2word.doc2bow(tokenize("This is a sample message of great duracell batteries"))

[(21, 1)]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
id2word.filter_extremes(no_below=5, no_above=0.95)
corpus = [id2word.doc2bow(text) for text in df['tokenize_text']]
lda = LdaMulticore(
    corpus=corpus,
    id2word=id2word,
    random_state=42,
    num_topics=10,
    passes=10,
    workers=4
)

In [ ]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
love tablet kid game old

------ Topic 1 ------
read kindl screen book app

------ Topic 2 ------
gift love purchas bought happi

------ Topic 3 ------
tablet us easi great love

------ Topic 4 ------
batteri brand work long price

------ Topic 5 ------
great good price product tablet

------ Topic 6 ------
amazon music sound speaker alexa

------ Topic 7 ------
long batteri work time expect

------ Topic 8 ------
kindl charg year replac bought

------ Topic 9 ------
need great item look review



In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.223310 -0.038436       1        1  18.821781
0     -0.206401  0.026689       2        1  17.201134
1     -0.115394 -0.026502       3        1  13.990417
5      0.138817  0.277512       4        1  10.150064
8     -0.089166 -0.129742       5        1   9.156316
3     -0.106860  0.119247       6        1   7.964512
2     -0.162668 -0.014420       7        1   5.955810
7      0.174652 -0.115774       8        1   5.791888
6      0.036624 -0.030014       9        1   5.684109
9      0.107086 -0.068560      10        1   5.283971, topic_info=         Term         Freq        Total Category  logprob  loglift
2     batteri  9708.000000  9708.000000  Default  30.0000  30.0000
1856   tablet  7639.000000  7639.000000  Default  29.0000  29.0000
33       good  5503.000000  5503.000000  Default  28.0000  28.0000
17      price  5082.000000  5082.000000  Default  27.0000  27.0000
20      great  8871.000000  8871.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
362      sure    76.150276   434.357666  Topic10  -5.3598   1.1993
53      power    78.364708   542.089661  Topic10  -5.3312   1.0064
240      case    81.941414   832.895874  Topic10  -5.2865   0.6216
39    purchas    88.131828  2619.554688  Topic10  -5.2137  -0.4514
379      deal    80.869972   809.454956  Topic10  -5.2997   0.6370

[775 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
871       1  0.986369         aa
54        1  0.913618        aaa
54        8  0.002735        aaa
54        9  0.013677        aaa
54       10  0.068385        aaa
...     ...       ...        ...
94        8  0.000888       year
94        9  0.008439       year
783       2  0.992503      young
2451      7  0.922625  youngster
2061     10  0.928192     zipper

[2235 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 2, 6, 9, 4, 3, 8, 7, 10])

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling